In [22]:
# 必要なパッケージのインストール

In [1]:
#!pip install pandas
#!pip install datapackage
#!pip install SQLAlchemy
#!pip install tableschema-sql

In [23]:
# 銘柄データを取得してSQLLiteで保存
# 参考（https://rainbow-engine.com/python-sqlalchemy-importcsv/）

In [119]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///periodic-table-datapackage_company.db')

In [120]:
from datapackage import Package
package = Package('https://datahub.io/core/s-and-p-500-companies/datapackage.json')
#package.resource_names

In [121]:
package.save(storage='sql', engine=engine)

Storage <Engine(sqlite:///periodic-table-datapackage_company.db)/None>

In [122]:
# 銘柄情報テーブルの内容確認

In [123]:
from sqlalchemy import Table, Column, Integer, String, MetaData
tbl = Table('constituents_csv', MetaData(), autoload_with=engine)

In [124]:
print(tbl.c.keys())

['Symbol', 'Name', 'Sector']


In [125]:
print(list(engine.execute('SELECT count() from constituents_csv')))

[(505,)]


In [126]:
# SQLで銘柄情報を取得

In [127]:
#print(list(engine.execute('SELECT * from constituents_csv where Name like \'%Bank%\'')))
print(list(engine.execute('SELECT Symbol from constituents_csv limit 10')))

[('MMM',), ('AOS',), ('ABT',), ('ABBV',), ('ABMD',), ('ACN',), ('ATVI',), ('ADBE',), ('AAP',), ('AMD',)]


In [ ]:
# DataFrame形式で作成

In [214]:
import pandas as pd
df = pd.DataFrame(list(engine.execute('SELECT * from constituents_csv')))
df.columns = ['Symbol', 'Name', 'Sector']
df.set_index("Symbol",inplace=True)

In [216]:
df.to_csv("./output/s_and_p_stock_list.csv")

In [33]:
# 追加情報付きのデータ取得

In [2]:
from datapackage import Package
package = Package('https://datahub.io/core/s-and-p-500-companies-financials/datapackage.json')

In [34]:
## 取得したデータ内容の確認とCS

In [17]:
package.resource_names
# package.get_resource('constituents-financials_csv').read()

['validation_report',
 'constituents_csv',
 'constituents-financials_csv',
 'constituents_json',
 'constituents-financials_json',
 's-and-p-500-companies-financials_zip',
 'constituents',
 'constituents-financials']

In [27]:
engine = create_engine('sqlite:///periodic-table-datapackage_company-wk.db')
package.save(storage='sql', engine=engine)

Storage <Engine(sqlite:///periodic-table-datapackage_company-wk.db)/None>

In [46]:
# 1970~2018年の株価データの確認

In [57]:
import pandas as pd
df = pd.read_csv('./resource_files/historical_stock_prices.csv.zip', sep=',', header=0)

In [58]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20973889 entries, 0 to 20973888
Data columns (total 8 columns):
ticker       object
open         float64
close        float64
adj_close    float64
low          float64
high         float64
volume       int64
date         object
dtypes: float64(5), int64(1), object(2)
memory usage: 1.3+ GB


In [60]:
df.head()

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14


In [61]:
df.tail()

,ticker,open,close,adj_close,low,high,volume,date
20973884,NZF,14.60,14.59,14.59,14.58,14.62,137500,2018-08-20
20973885,NZF,14.60,14.58,14.58,14.57,14.61,151200,2018-08-21
20973886,NZF,14.58,14.59,14.59,14.57,14.63,185400,2018-08-22
20973887,NZF,14.60,14.57,14.57,14.57,14.64,135600,2018-08-23
20973888,NZF,14.60,14.69,14.69,14.59,14.69,180900,2018-08-24


In [64]:
df.agg({"date":[min,max]})

,date
min,1970-01-02
max,2018-08-24


In [63]:
df.query("ticker=='MMM'").agg({"date":[min,max]})

,date
min,1970-01-02
max,2018-08-24


In [65]:
# Macrotrendsからデータを抽出

In [66]:
## クローラーの準備

In [67]:
### 必要なパッケージのインストール

In [90]:
# !pip install selenium

In [16]:
import os
import pandas as pd

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
# headless optionの設定
options = Options()
options.add_argument("--headless");
# driverの定義
driver = webdriver.Chrome(executable_path="./resource_files/chromedriver", chrome_options=options)

/usr/local/var/pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [3]:
## Macrotrendsへのクローラーの設定

In [4]:
url = "https://www.macrotrends.net/stocks/charts/"

In [5]:
# MacrotrendsのTopページへアクセス
driver.get(os.path.join(url, "MMM"))
target_url = driver.current_url

# 以降、Topページからの検索はjQueryでうまく実行できないため、銘柄ごとのURLを取得して、必要な情報のパスを追加してアクセスする

In [6]:
## データ取得処理のサンプル

In [298]:
## revenue
driver.get(os.path.join(target_url, "revenue"))
elements = driver.find_elements_by_css_selector('#style-1 > .col-xs-6:nth-child(2) > .historical_data_table.table > tbody > tr')

# サンプルの取得例
elements[0].text.split(" $")

['2020-09-30', '8,350']

In [307]:
list(map(lambda f: f.text, elements[0].find_elements_by_css_selector('td')))

['2020-09-30', '$8,350']

In [240]:
## revenue (rule 2)
driver.get(os.path.join(target_url, "revenue"))
elements = driver.find_elements_by_css_selector('#style-1 > .col-xs-6:nth-child(2) > .historical_data_table.table > tbody > tr')

# サンプルの取得例
# elements[0].text.split(" $")

# サンプルdfの生成例
df =pd.DataFrame([], columns=["SYMBOL", "INDICATOR", "DATE", "VALUE", "UNIT"])
for i in range(0, len(elements)):
    indicator = "cost-goods-sold"
    date_val, val = elements[i].text.split(" $")
    df = df.append({'SYMBOL': "MMM", 'INDICATOR': indicator, 'DATE': date_val, 'VALUE': val, "UNIT": "$"}, ignore_index=True)
df.tail()

,SYMBOL,INDICATOR,DATE,VALUE,UNIT
58,MMM,cost-goods-sold,2006-03-31,"5,595",$
59,MMM,cost-goods-sold,2005-12-31,"5,325",$
60,MMM,cost-goods-sold,2005-09-30,"5,382",$
61,MMM,cost-goods-sold,2005-06-30,"5,294",$
62,MMM,cost-goods-sold,2005-03-31,"5,166",$


In [242]:
## gross-margin (rule 1)
driver.get(os.path.join(target_url, "gross-margin"))
elements = driver.find_elements_by_css_selector('#style-1 > .table:nth-child(1) > tbody > tr')

# サンプルの取得例
# elements[0].text.split(" ")

# # サンプルdfの生成例
df =pd.DataFrame([], columns=["SYMBOL", "INDICATOR", "DATE", "VALUE", "UNIT"])
for i in range(0, len(elements)):
    indicator = "cost-goods-sold"
    date_val, _, _, val = elements[i].text.split(" ")
    val = val.split("%")[0]
    df = df.append({'SYMBOL': "MMM", 'INDICATOR': indicator, 'DATE': date_val, 'VALUE': val, "UNIT": "%"}, ignore_index=True)
df.tail()

,SYMBOL,INDICATOR,DATE,VALUE,UNIT
55,MMM,cost-goods-sold,2006-12-31,48.90,%
56,MMM,cost-goods-sold,2006-09-30,50.27,%
57,MMM,cost-goods-sold,2006-06-30,50.80,%
58,MMM,cost-goods-sold,2006-03-31,51.01,%
59,MMM,cost-goods-sold,2005-12-31,50.83,%


In [7]:
## まとめて取得するための設定

In [8]:
macrotrends_scrapying_rules = [
    {
        "CATEGORY": "Revenue&Profit",
        "INDICATORS": [
            {"INDICATOR": "revenue", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "gross-profit", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "operating-income", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "ebitda", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "net-income", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "eps-earnings-per-share-diluted", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "shares-outstanding", "RULE": 2, "UNIT": ""}]
    },
    {
        "CATEGORY": "Assets&Liabilityies",
        "INDICATORS": [
            {"INDICATOR": "total-assets", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "cash-on-hand", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "long-term-debt", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "total-liabilities", "RULE": 2, "UNIT": "$"}
            , {"INDICATOR": "total-share-holder-equity", "RULE": 2, "UNIT": "$"}]
    },
    {
        "CATEGORY": "Margins",
        "INDICATORS": [
            {"INDICATOR": "profit-margins", "RULE": 2, "UNIT": "%"}
            , {"INDICATOR": "gross-margin", "RULE": 1, "UNIT": "%"}
            , {"INDICATOR": "operating-margin", "RULE": 1, "UNIT": "%"}
            , {"INDICATOR": "ebitda-margin", "RULE": 1, "UNIT": "%"}
            , {"INDICATOR": "pre-tax-profit-margin", "RULE": 1, "UNIT": "%"}
            , {"INDICATOR": "net-profit-margin", "RULE": 1, "UNIT": "%"}]
    },
    {
        "CATEGORY": "Price_Ratios",
        "INDICATORS": [
            {"INDICATOR": "pe-ratio", "RULE": 1, "UNIT": ""}
            , {"INDICATOR": "price-sales", "RULE": 1, "UNIT": ""}
            , {"INDICATOR": "price-book", "RULE": 1, "UNIT": ""}
            , {"INDICATOR": "price-fcf", "RULE": 1, "UNIT": ""}]
    },
    {
        "CATEGORY": "Other_Ratios",
        "INDICATORS": [
            {"INDICATOR": "current-ratio", "RULE": 1, "UNIT": ""}
            , {"INDICATOR": "quick-ratio", "RULE": 1, "UNIT": ""}
            , {"INDICATOR": "debt-equity-ratio", "RULE": 1, "UNIT": ""}
            , {"INDICATOR": "roe", "RULE": 1, "UNIT": "%"}
            , {"INDICATOR": "roa", "RULE": 1, "UNIT": "%"}
            , {"INDICATOR": "roi", "RULE": 1, "UNIT": "%"}
            , {"INDICATOR": "return-on-tangible-equity", "RULE": 1, "UNIT": "%"}]
    }
]

In [9]:
# S&P銘柄の情報をmacrotrendsから取得する関数を定義

In [10]:
## 必要なパッケージのインストール
import numpy as np
import time

In [11]:
# 銘柄の情報を引数のDataFrameへ登録
## dfはのcolumnsは、["SYMBOL", "INDICATOR", "DATE", "VALUE", "UNIT"]を想定。（それ以外の場合はエラー）
def addMacrotrendsInfoBySymbol(symbol, driver, macrotrends_rules):
    df =pd.DataFrame([], columns=["SYMBOL", "INDICATOR", "DATE", "VALUE", "UNIT"])
    base_url = "https://www.macrotrends.net/stocks/charts/"
    driver.get(os.path.join(base_url, symbol))
    target_url = driver.current_url
    
    start = time.time()
    rap_start = start
    print("# [START] symbol: {0}".format(symbol))
    for i in range(0, len(macrotrends_rules)):
        for j in range(0, len(macrotrends_rules[i]["INDICATORS"])):
            indicator, rule, unit = macrotrends_rules[i]["INDICATORS"][j].values()
            if rule == 1:
                driver.get(os.path.join(target_url, indicator))
                elements = driver.find_elements_by_css_selector('#style-1 > .table:nth-child(1) > tbody > tr')
                for k in range(0, len(elements)):
                    date_val, _, _, val = list(map(lambda f: f.text, elements[k].find_elements_by_css_selector('td')))
                    val = val.replace(unit, "")
                    df = df.append({'SYMBOL': symbol, 'INDICATOR': indicator, 'DATE': date_val, 'VALUE': val, "UNIT": unit}, ignore_index=True)
            else:
                if rule == 2:
                    driver.get(os.path.join(target_url, indicator))
                    elements = driver.find_elements_by_css_selector('#style-1 > .col-xs-6:nth-child(2) > .historical_data_table.table > tbody > tr')
                    for k in range(0, len(elements)):
                        date_val, val = list(map(lambda f: f.text, elements[k].find_elements_by_css_selector('td')))
                        val = val.replace(unit, "")
                        df = df.append({'SYMBOL': symbol, 'INDICATOR': indicator, 'DATE': date_val, 'VALUE': val, "UNIT": unit}, ignore_index=True)
                else:
                    raise Exception("macrotrends_rules is incorrect. rule: {0}, {1}".format(indicator, rule))
            
#             elapsed_time = time.time() - rap_start
#             print ("## [RAP] indicator: {0} / elapsed_time: {1:.2f}".format(indicator, elapsed_time) + "[sec]")
#             rap_start = time.time()
    print("# [END] symbol: {0} / total_time: {1:.2f}".format(symbol, time.time() - start) + "[sec]")
    return df

In [12]:
# S&Pのsymbolを取得

In [17]:
df_s_and_p_stock_list = pd.read_csv("./output/s_and_p_stock_list.csv", sep=",", header=0)

In [18]:
s_and_p_stock_list = df_s_and_p_stock_list["Symbol"].unique()

In [19]:
len(s_and_p_stock_list)

505

In [374]:
# 1970~2018年のストックデータに含まれる銘柄の確認

In [375]:
df_historical_stocks = pd.read_csv("./resource_files/historical_stock_prices.csv.zip", sep=",", header=0)

In [376]:
df_historical_stocks.head()

,ticker,open,close,adj_close,low,high,volume,date
0,AHH,11.50,11.58,8.493155,11.25,11.68,4633900,2013-05-08
1,AHH,11.66,11.55,8.471151,11.50,11.66,275800,2013-05-09
2,AHH,11.55,11.60,8.507822,11.50,11.60,277100,2013-05-10
3,AHH,11.63,11.65,8.544494,11.55,11.65,147400,2013-05-13
4,AHH,11.60,11.53,8.456484,11.50,11.60,184100,2013-05-14


In [377]:
df_historical_stocks[["ticker"]] \
    .drop_duplicates() \
    .query("ticker in @s_and_p_stock_list") \
    .shape

(484, 1)

In [344]:
# カテゴリごとにデータを取得

In [20]:
## 事前定義のオブジェクトを準備
options = Options()
options.add_argument("--headless");
driver = webdriver.Chrome(executable_path="./resource_files/chromedriver", chrome_options=options)
# df =pd.DataFrame([], columns=["SYMBOL", "INDICATOR", "DATE", "VALUE", "UNIT"])

/usr/local/var/pyenv/versions/3.7.3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: use options instead of chrome_options
  after removing the cwd from sys.path.


In [403]:
index = 0
rules = [macrotrends_scrapying_rules[index]]
category = rules[0]["CATEGORY"]

## 実行
for i in range(0, len(s_and_p_stock_list)):
    df = df.append(addMacrotrendsInfoBySymbol(s_and_p_stock_list[i], driver, rules))
df.to_csv("./output/macrotrends_{0}.csv".format(category))

# [START] symbol: MMM
# [END] symbol: MMM / total_time: 55.41[sec]
# [START] symbol: AOS
# [END] symbol: AOS / total_time: 45.21[sec]
# [START] symbol: ABT
# [END] symbol: ABT / total_time: 49.76[sec]
# [START] symbol: ABBV
# [END] symbol: ABBV / total_time: 37.92[sec]
# [START] symbol: ABMD
# [END] symbol: ABMD / total_time: 46.95[sec]
# [START] symbol: ACN
# [END] symbol: ACN / total_time: 53.50[sec]
# [START] symbol: ATVI
# [END] symbol: ATVI / total_time: 50.91[sec]
# [START] symbol: ADBE
# [END] symbol: ADBE / total_time: 45.25[sec]
# [START] symbol: AAP
# [END] symbol: AAP / total_time: 45.68[sec]
# [START] symbol: AMD
# [END] symbol: AMD / total_time: 45.41[sec]
# [START] symbol: AES
# [END] symbol: AES / total_time: 53.13[sec]
# [START] symbol: AFL
# [END] symbol: AFL / total_time: 52.27[sec]
# [START] symbol: A
# [END] symbol: A / total_time: 54.40[sec]
# [START] symbol: APD
# [END] symbol: APD / total_time: 50.52[sec]
# [START] symbol: AKAM
# [END] symbol: AKAM / total_time: 

# [START] symbol: STZ
# [END] symbol: STZ / total_time: 53.60[sec]
# [START] symbol: CPRT
# [END] symbol: CPRT / total_time: 51.84[sec]
# [START] symbol: GLW
# [END] symbol: GLW / total_time: 52.18[sec]
# [START] symbol: CTVA
# [END] symbol: CTVA / total_time: 19.12[sec]
# [START] symbol: COST
# [END] symbol: COST / total_time: 52.55[sec]
# [START] symbol: COTY
# [END] symbol: COTY / total_time: 39.90[sec]
# [START] symbol: CCI
# [END] symbol: CCI / total_time: 53.52[sec]
# [START] symbol: CSX
# [END] symbol: CSX / total_time: 49.68[sec]
# [START] symbol: CMI
# [END] symbol: CMI / total_time: 53.43[sec]
# [START] symbol: CVS
# [END] symbol: CVS / total_time: 48.86[sec]
# [START] symbol: DHI
# [END] symbol: DHI / total_time: 49.01[sec]
# [START] symbol: DHR
# [END] symbol: DHR / total_time: 50.49[sec]
# [START] symbol: DRI
# [END] symbol: DRI / total_time: 50.76[sec]
# [START] symbol: DVA
# [END] symbol: DVA / total_time: 52.02[sec]
# [START] symbol: DE
# [END] symbol: DE / total_time: 

# [START] symbol: INCY
# [END] symbol: INCY / total_time: 52.41[sec]
# [START] symbol: IR
# [END] symbol: IR / total_time: 29.42[sec]
# [START] symbol: INTC
# [END] symbol: INTC / total_time: 48.81[sec]
# [START] symbol: ICE
# [END] symbol: ICE / total_time: 49.03[sec]
# [START] symbol: IBM
# [END] symbol: IBM / total_time: 50.32[sec]
# [START] symbol: IFF
# [END] symbol: IFF / total_time: 45.98[sec]
# [START] symbol: IP
# [END] symbol: IP / total_time: 47.99[sec]
# [START] symbol: IPG
# [END] symbol: IPG / total_time: 49.33[sec]
# [START] symbol: INTU
# [END] symbol: INTU / total_time: 49.10[sec]
# [START] symbol: ISRG
# [END] symbol: ISRG / total_time: 51.07[sec]
# [START] symbol: IVZ
# [END] symbol: IVZ / total_time: 50.98[sec]
# [START] symbol: IPGP
# [END] symbol: IPGP / total_time: 48.97[sec]
# [START] symbol: IQV
# [END] symbol: IQV / total_time: 36.04[sec]
# [START] symbol: IRM
# [END] symbol: IRM / total_time: 47.75[sec]
# [START] symbol: JBHT
# [END] symbol: JBHT / total_time

# [START] symbol: PFE
# [END] symbol: PFE / total_time: 43.72[sec]
# [START] symbol: PM
# [END] symbol: PM / total_time: 45.98[sec]
# [START] symbol: PSX
# [END] symbol: PSX / total_time: 39.89[sec]
# [START] symbol: PNW
# [END] symbol: PNW / total_time: 47.42[sec]
# [START] symbol: PXD
# [END] symbol: PXD / total_time: 47.77[sec]
# [START] symbol: PNC
# [END] symbol: PNC / total_time: 49.07[sec]
# [START] symbol: PPG
# [END] symbol: PPG / total_time: 48.21[sec]
# [START] symbol: PPL
# [END] symbol: PPL / total_time: 50.93[sec]
# [START] symbol: PFG
# [END] symbol: PFG / total_time: 49.91[sec]
# [START] symbol: PG
# [END] symbol: PG / total_time: 49.47[sec]
# [START] symbol: PGR
# [END] symbol: PGR / total_time: 49.42[sec]
# [START] symbol: PLD
# [END] symbol: PLD / total_time: 50.19[sec]
# [START] symbol: PRU
# [END] symbol: PRU / total_time: 51.27[sec]
# [START] symbol: PEG
# [END] symbol: PEG / total_time: 49.30[sec]
# [START] symbol: PSA
# [END] symbol: PSA / total_time: 57.91[sec]

# [END] symbol: WDC / total_time: 53.73[sec]
# [START] symbol: WU
# [END] symbol: WU / total_time: 49.77[sec]
# [START] symbol: WRK
# [END] symbol: WRK / total_time: 50.15[sec]
# [START] symbol: WY
# [END] symbol: WY / total_time: 49.84[sec]
# [START] symbol: WHR
# [END] symbol: WHR / total_time: 50.50[sec]
# [START] symbol: WMB
# [END] symbol: WMB / total_time: 50.85[sec]
# [START] symbol: WLTW
# [END] symbol: WLTW / total_time: 50.59[sec]
# [START] symbol: WYNN
# [END] symbol: WYNN / total_time: 49.57[sec]
# [START] symbol: XEL
# [END] symbol: XEL / total_time: 50.74[sec]
# [START] symbol: XRX
# [END] symbol: XRX / total_time: 51.90[sec]
# [START] symbol: XLNX
# [END] symbol: XLNX / total_time: 48.49[sec]
# [START] symbol: XYL
# [END] symbol: XYL / total_time: 42.79[sec]
# [START] symbol: YUM
# [END] symbol: YUM / total_time: 51.52[sec]
# [START] symbol: ZBRA
# [END] symbol: ZBRA / total_time: 49.95[sec]
# [START] symbol: ZBH
# [END] symbol: ZBH / total_time: 51.00[sec]
# [START] sym

In [404]:
index = 1
rules = [macrotrends_scrapying_rules[index]]
category = rules[0]["CATEGORY"]

## 実行
for i in range(0, len(s_and_p_stock_list)):
    df = df.append(addMacrotrendsInfoBySymbol(s_and_p_stock_list[i], driver, rules))
df.to_csv("./output/macrotrends_{0}.csv".format(category))

# [START] symbol: MMM
# [END] symbol: MMM / total_time: 36.67[sec]
# [START] symbol: AOS
# [END] symbol: AOS / total_time: 35.27[sec]
# [START] symbol: ABT
# [END] symbol: ABT / total_time: 35.58[sec]
# [START] symbol: ABBV
# [END] symbol: ABBV / total_time: 27.41[sec]
# [START] symbol: ABMD
# [END] symbol: ABMD / total_time: 35.20[sec]
# [START] symbol: ACN
# [END] symbol: ACN / total_time: 36.49[sec]
# [START] symbol: ATVI
# [END] symbol: ATVI / total_time: 36.89[sec]
# [START] symbol: ADBE
# [END] symbol: ADBE / total_time: 35.04[sec]
# [START] symbol: AAP
# [END] symbol: AAP / total_time: 35.58[sec]
# [START] symbol: AMD
# [END] symbol: AMD / total_time: 34.38[sec]
# [START] symbol: AES
# [END] symbol: AES / total_time: 33.66[sec]
# [START] symbol: AFL
# [END] symbol: AFL / total_time: 33.91[sec]
# [START] symbol: A
# [END] symbol: A / total_time: 34.86[sec]
# [START] symbol: APD
# [END] symbol: APD / total_time: 34.72[sec]
# [START] symbol: AKAM
# [END] symbol: AKAM / total_time: 

# [START] symbol: STZ
# [END] symbol: STZ / total_time: 36.04[sec]
# [START] symbol: CPRT
# [END] symbol: CPRT / total_time: 35.44[sec]
# [START] symbol: GLW
# [END] symbol: GLW / total_time: 37.78[sec]
# [START] symbol: CTVA
# [END] symbol: CTVA / total_time: 15.37[sec]
# [START] symbol: COST
# [END] symbol: COST / total_time: 36.14[sec]
# [START] symbol: COTY
# [END] symbol: COTY / total_time: 27.51[sec]
# [START] symbol: CCI
# [END] symbol: CCI / total_time: 37.19[sec]
# [START] symbol: CSX
# [END] symbol: CSX / total_time: 38.19[sec]
# [START] symbol: CMI
# [END] symbol: CMI / total_time: 38.31[sec]
# [START] symbol: CVS
# [END] symbol: CVS / total_time: 35.45[sec]
# [START] symbol: DHI
# [END] symbol: DHI / total_time: 36.08[sec]
# [START] symbol: DHR
# [END] symbol: DHR / total_time: 38.58[sec]
# [START] symbol: DRI
# [END] symbol: DRI / total_time: 35.05[sec]
# [START] symbol: DVA
# [END] symbol: DVA / total_time: 34.80[sec]
# [START] symbol: DE
# [END] symbol: DE / total_time: 

# [START] symbol: INCY
# [END] symbol: INCY / total_time: 35.83[sec]
# [START] symbol: IR
# [END] symbol: IR / total_time: 21.55[sec]
# [START] symbol: INTC
# [END] symbol: INTC / total_time: 34.48[sec]
# [START] symbol: ICE
# [END] symbol: ICE / total_time: 35.27[sec]
# [START] symbol: IBM
# [END] symbol: IBM / total_time: 34.28[sec]
# [START] symbol: IFF
# [END] symbol: IFF / total_time: 35.98[sec]
# [START] symbol: IP
# [END] symbol: IP / total_time: 32.23[sec]
# [START] symbol: IPG
# [END] symbol: IPG / total_time: 32.81[sec]
# [START] symbol: INTU
# [END] symbol: INTU / total_time: 37.15[sec]
# [START] symbol: ISRG
# [END] symbol: ISRG / total_time: 35.44[sec]
# [START] symbol: IVZ
# [END] symbol: IVZ / total_time: 34.90[sec]
# [START] symbol: IPGP
# [END] symbol: IPGP / total_time: 34.51[sec]
# [START] symbol: IQV
# [END] symbol: IQV / total_time: 29.00[sec]
# [START] symbol: IRM
# [END] symbol: IRM / total_time: 35.36[sec]
# [START] symbol: JBHT
# [END] symbol: JBHT / total_time

# [START] symbol: PFE
# [END] symbol: PFE / total_time: 37.69[sec]
# [START] symbol: PM
# [END] symbol: PM / total_time: 32.57[sec]
# [START] symbol: PSX
# [END] symbol: PSX / total_time: 28.71[sec]
# [START] symbol: PNW
# [END] symbol: PNW / total_time: 36.63[sec]
# [START] symbol: PXD
# [END] symbol: PXD / total_time: 35.91[sec]
# [START] symbol: PNC
# [END] symbol: PNC / total_time: 35.74[sec]
# [START] symbol: PPG
# [END] symbol: PPG / total_time: 36.44[sec]
# [START] symbol: PPL
# [END] symbol: PPL / total_time: 35.50[sec]
# [START] symbol: PFG
# [END] symbol: PFG / total_time: 36.99[sec]
# [START] symbol: PG
# [END] symbol: PG / total_time: 34.49[sec]
# [START] symbol: PGR
# [END] symbol: PGR / total_time: 33.95[sec]
# [START] symbol: PLD
# [END] symbol: PLD / total_time: 35.84[sec]
# [START] symbol: PRU
# [END] symbol: PRU / total_time: 33.67[sec]
# [START] symbol: PEG
# [END] symbol: PEG / total_time: 34.92[sec]
# [START] symbol: PSA
# [END] symbol: PSA / total_time: 34.66[sec]

# [END] symbol: WDC / total_time: 30.98[sec]
# [START] symbol: WU
# [END] symbol: WU / total_time: 33.36[sec]
# [START] symbol: WRK
# [END] symbol: WRK / total_time: 35.81[sec]
# [START] symbol: WY
# [END] symbol: WY / total_time: 35.16[sec]
# [START] symbol: WHR
# [END] symbol: WHR / total_time: 34.12[sec]
# [START] symbol: WMB
# [END] symbol: WMB / total_time: 34.41[sec]
# [START] symbol: WLTW
# [END] symbol: WLTW / total_time: 35.19[sec]
# [START] symbol: WYNN
# [END] symbol: WYNN / total_time: 39.63[sec]
# [START] symbol: XEL
# [END] symbol: XEL / total_time: 44.43[sec]
# [START] symbol: XRX
# [END] symbol: XRX / total_time: 37.79[sec]
# [START] symbol: XLNX
# [END] symbol: XLNX / total_time: 39.78[sec]
# [START] symbol: XYL
# [END] symbol: XYL / total_time: 33.42[sec]
# [START] symbol: YUM
# [END] symbol: YUM / total_time: 37.82[sec]
# [START] symbol: ZBRA
# [END] symbol: ZBRA / total_time: 36.11[sec]
# [START] symbol: ZBH
# [END] symbol: ZBH / total_time: 34.86[sec]
# [START] sym

In [405]:
index = 2
rules = [macrotrends_scrapying_rules[index]]
category = rules[0]["CATEGORY"]

## 実行
for i in range(0, len(s_and_p_stock_list)):
    df = df.append(addMacrotrendsInfoBySymbol(s_and_p_stock_list[i], driver, rules))
df.to_csv("./output/macrotrends_{0}.csv".format(category))

# [START] symbol: MMM
# [END] symbol: MMM / total_time: 52.41[sec]
# [START] symbol: AOS
# [END] symbol: AOS / total_time: 49.47[sec]
# [START] symbol: ABT
# [END] symbol: ABT / total_time: 59.65[sec]
# [START] symbol: ABBV
# [END] symbol: ABBV / total_time: 35.37[sec]
# [START] symbol: ABMD
# [END] symbol: ABMD / total_time: 49.79[sec]
# [START] symbol: ACN
# [END] symbol: ACN / total_time: 53.00[sec]
# [START] symbol: ATVI
# [END] symbol: ATVI / total_time: 44.14[sec]
# [START] symbol: ADBE
# [END] symbol: ADBE / total_time: 47.29[sec]
# [START] symbol: AAP
# [END] symbol: AAP / total_time: 46.55[sec]
# [START] symbol: AMD
# [END] symbol: AMD / total_time: 45.64[sec]
# [START] symbol: AES
# [END] symbol: AES / total_time: 44.49[sec]
# [START] symbol: AFL
# [END] symbol: AFL / total_time: 46.89[sec]
# [START] symbol: A
# [END] symbol: A / total_time: 48.36[sec]
# [START] symbol: APD
# [END] symbol: APD / total_time: 49.07[sec]
# [START] symbol: AKAM
# [END] symbol: AKAM / total_time: 

# [START] symbol: STZ
# [END] symbol: STZ / total_time: 47.89[sec]
# [START] symbol: CPRT
# [END] symbol: CPRT / total_time: 48.42[sec]
# [START] symbol: GLW
# [END] symbol: GLW / total_time: 47.85[sec]
# [START] symbol: CTVA
# [END] symbol: CTVA / total_time: 17.20[sec]
# [START] symbol: COST
# [END] symbol: COST / total_time: 45.36[sec]
# [START] symbol: COTY
# [END] symbol: COTY / total_time: 29.62[sec]
# [START] symbol: CCI
# [END] symbol: CCI / total_time: 49.67[sec]
# [START] symbol: CSX
# [END] symbol: CSX / total_time: 49.02[sec]
# [START] symbol: CMI
# [END] symbol: CMI / total_time: 44.67[sec]
# [START] symbol: CVS
# [END] symbol: CVS / total_time: 47.26[sec]
# [START] symbol: DHI
# [END] symbol: DHI / total_time: 48.76[sec]
# [START] symbol: DHR
# [END] symbol: DHR / total_time: 52.21[sec]
# [START] symbol: DRI
# [END] symbol: DRI / total_time: 49.60[sec]
# [START] symbol: DVA
# [END] symbol: DVA / total_time: 49.34[sec]
# [START] symbol: DE
# [END] symbol: DE / total_time: 

# [START] symbol: INCY
# [END] symbol: INCY / total_time: 46.96[sec]
# [START] symbol: IR
# [END] symbol: IR / total_time: 23.88[sec]
# [START] symbol: INTC
# [END] symbol: INTC / total_time: 49.90[sec]
# [START] symbol: ICE
# [END] symbol: ICE / total_time: 48.37[sec]
# [START] symbol: IBM
# [END] symbol: IBM / total_time: 46.79[sec]
# [START] symbol: IFF
# [END] symbol: IFF / total_time: 45.20[sec]
# [START] symbol: IP
# [END] symbol: IP / total_time: 47.97[sec]
# [START] symbol: IPG
# [END] symbol: IPG / total_time: 47.49[sec]
# [START] symbol: INTU
# [END] symbol: INTU / total_time: 47.71[sec]
# [START] symbol: ISRG
# [END] symbol: ISRG / total_time: 49.19[sec]
# [START] symbol: IVZ
# [END] symbol: IVZ / total_time: 49.02[sec]
# [START] symbol: IPGP
# [END] symbol: IPGP / total_time: 46.66[sec]
# [START] symbol: IQV
# [END] symbol: IQV / total_time: 33.87[sec]
# [START] symbol: IRM
# [END] symbol: IRM / total_time: 47.99[sec]
# [START] symbol: JBHT
# [END] symbol: JBHT / total_time

# [START] symbol: PFE
# [END] symbol: PFE / total_time: 50.37[sec]
# [START] symbol: PM
# [END] symbol: PM / total_time: 46.75[sec]
# [START] symbol: PSX
# [END] symbol: PSX / total_time: 35.50[sec]
# [START] symbol: PNW
# [END] symbol: PNW / total_time: 51.07[sec]
# [START] symbol: PXD
# [END] symbol: PXD / total_time: 51.22[sec]
# [START] symbol: PNC
# [END] symbol: PNC / total_time: 49.44[sec]
# [START] symbol: PPG
# [END] symbol: PPG / total_time: 49.46[sec]
# [START] symbol: PPL
# [END] symbol: PPL / total_time: 51.78[sec]
# [START] symbol: PFG
# [END] symbol: PFG / total_time: 50.56[sec]
# [START] symbol: PG
# [END] symbol: PG / total_time: 49.77[sec]
# [START] symbol: PGR
# [END] symbol: PGR / total_time: 49.23[sec]
# [START] symbol: PLD
# [END] symbol: PLD / total_time: 50.78[sec]
# [START] symbol: PRU
# [END] symbol: PRU / total_time: 48.42[sec]
# [START] symbol: PEG
# [END] symbol: PEG / total_time: 48.92[sec]
# [START] symbol: PSA
# [END] symbol: PSA / total_time: 49.61[sec]

# [END] symbol: WDC / total_time: 47.95[sec]
# [START] symbol: WU
# [END] symbol: WU / total_time: 47.92[sec]
# [START] symbol: WRK
# [END] symbol: WRK / total_time: 37.70[sec]
# [START] symbol: WY
# [END] symbol: WY / total_time: 48.01[sec]
# [START] symbol: WHR
# [END] symbol: WHR / total_time: 50.09[sec]
# [START] symbol: WMB
# [END] symbol: WMB / total_time: 50.90[sec]
# [START] symbol: WLTW
# [END] symbol: WLTW / total_time: 51.77[sec]
# [START] symbol: WYNN
# [END] symbol: WYNN / total_time: 49.42[sec]
# [START] symbol: XEL
# [END] symbol: XEL / total_time: 50.32[sec]
# [START] symbol: XRX
# [END] symbol: XRX / total_time: 48.14[sec]
# [START] symbol: XLNX
# [END] symbol: XLNX / total_time: 47.49[sec]
# [START] symbol: XYL
# [END] symbol: XYL / total_time: 39.71[sec]
# [START] symbol: YUM
# [END] symbol: YUM / total_time: 47.73[sec]
# [START] symbol: ZBRA
# [END] symbol: ZBRA / total_time: 47.76[sec]
# [START] symbol: ZBH
# [END] symbol: ZBH / total_time: 51.28[sec]
# [START] sym

In [406]:
index = 3
rules = [macrotrends_scrapying_rules[index]]
category = rules[0]["CATEGORY"]

## 実行
for i in range(0, len(s_and_p_stock_list)):
    df = df.append(addMacrotrendsInfoBySymbol(s_and_p_stock_list[i], driver, rules))
df.to_csv("./output/macrotrends_{0}.csv".format(category))

# [START] symbol: MMM
# [END] symbol: MMM / total_time: 39.17[sec]
# [START] symbol: AOS
# [END] symbol: AOS / total_time: 37.02[sec]
# [START] symbol: ABT
# [END] symbol: ABT / total_time: 35.57[sec]
# [START] symbol: ABBV
# [END] symbol: ABBV / total_time: 21.35[sec]
# [START] symbol: ABMD
# [END] symbol: ABMD / total_time: 36.80[sec]
# [START] symbol: ACN
# [END] symbol: ACN / total_time: 37.45[sec]
# [START] symbol: ATVI
# [END] symbol: ATVI / total_time: 36.03[sec]
# [START] symbol: ADBE
# [END] symbol: ADBE / total_time: 36.50[sec]
# [START] symbol: AAP
# [END] symbol: AAP / total_time: 40.37[sec]
# [START] symbol: AMD
# [END] symbol: AMD / total_time: 37.64[sec]
# [START] symbol: AES
# [END] symbol: AES / total_time: 35.68[sec]
# [START] symbol: AFL
# [END] symbol: AFL / total_time: 41.94[sec]
# [START] symbol: A
# [END] symbol: A / total_time: 37.26[sec]
# [START] symbol: APD
# [END] symbol: APD / total_time: 35.57[sec]
# [START] symbol: AKAM
# [END] symbol: AKAM / total_time: 

# [START] symbol: STZ
# [END] symbol: STZ / total_time: 37.55[sec]
# [START] symbol: CPRT
# [END] symbol: CPRT / total_time: 36.65[sec]
# [START] symbol: GLW
# [END] symbol: GLW / total_time: 38.77[sec]
# [START] symbol: CTVA
# [END] symbol: CTVA / total_time: 13.86[sec]
# [START] symbol: COST
# [END] symbol: COST / total_time: 39.74[sec]
# [START] symbol: COTY
# [END] symbol: COTY / total_time: 26.18[sec]
# [START] symbol: CCI
# [END] symbol: CCI / total_time: 38.80[sec]
# [START] symbol: CSX
# [END] symbol: CSX / total_time: 36.56[sec]
# [START] symbol: CMI
# [END] symbol: CMI / total_time: 34.64[sec]
# [START] symbol: CVS
# [END] symbol: CVS / total_time: 38.36[sec]
# [START] symbol: DHI
# [END] symbol: DHI / total_time: 36.54[sec]
# [START] symbol: DHR
# [END] symbol: DHR / total_time: 37.47[sec]
# [START] symbol: DRI
# [END] symbol: DRI / total_time: 38.22[sec]
# [START] symbol: DVA
# [END] symbol: DVA / total_time: 37.17[sec]
# [START] symbol: DE
# [END] symbol: DE / total_time: 

# [START] symbol: INCY
# [END] symbol: INCY / total_time: 39.15[sec]
# [START] symbol: IR
# [END] symbol: IR / total_time: 19.14[sec]
# [START] symbol: INTC
# [END] symbol: INTC / total_time: 40.47[sec]
# [START] symbol: ICE
# [END] symbol: ICE / total_time: 36.61[sec]
# [START] symbol: IBM
# [END] symbol: IBM / total_time: 38.14[sec]
# [START] symbol: IFF
# [END] symbol: IFF / total_time: 37.15[sec]
# [START] symbol: IP
# [END] symbol: IP / total_time: 37.42[sec]
# [START] symbol: IPG
# [END] symbol: IPG / total_time: 35.73[sec]
# [START] symbol: INTU
# [END] symbol: INTU / total_time: 36.54[sec]
# [START] symbol: ISRG
# [END] symbol: ISRG / total_time: 38.17[sec]
# [START] symbol: IVZ
# [END] symbol: IVZ / total_time: 38.45[sec]
# [START] symbol: IPGP
# [END] symbol: IPGP / total_time: 34.35[sec]
# [START] symbol: IQV
# [END] symbol: IQV / total_time: 26.52[sec]
# [START] symbol: IRM
# [END] symbol: IRM / total_time: 36.34[sec]
# [START] symbol: JBHT
# [END] symbol: JBHT / total_time

# [START] symbol: PFE
# [END] symbol: PFE / total_time: 36.48[sec]
# [START] symbol: PM
# [END] symbol: PM / total_time: 34.09[sec]
# [START] symbol: PSX
# [END] symbol: PSX / total_time: 24.42[sec]
# [START] symbol: PNW
# [END] symbol: PNW / total_time: 34.66[sec]
# [START] symbol: PXD
# [END] symbol: PXD / total_time: 39.59[sec]
# [START] symbol: PNC
# [END] symbol: PNC / total_time: 38.78[sec]
# [START] symbol: PPG
# [END] symbol: PPG / total_time: 36.66[sec]
# [START] symbol: PPL
# [END] symbol: PPL / total_time: 37.75[sec]
# [START] symbol: PFG
# [END] symbol: PFG / total_time: 40.85[sec]
# [START] symbol: PG
# [END] symbol: PG / total_time: 38.65[sec]
# [START] symbol: PGR
# [END] symbol: PGR / total_time: 37.46[sec]
# [START] symbol: PLD
# [END] symbol: PLD / total_time: 39.33[sec]
# [START] symbol: PRU
# [END] symbol: PRU / total_time: 38.46[sec]
# [START] symbol: PEG
# [END] symbol: PEG / total_time: 36.74[sec]
# [START] symbol: PSA
# [END] symbol: PSA / total_time: 40.52[sec]

# [END] symbol: WDC / total_time: 35.33[sec]
# [START] symbol: WU
# [END] symbol: WU / total_time: 36.75[sec]
# [START] symbol: WRK
# [END] symbol: WRK / total_time: 21.39[sec]
# [START] symbol: WY
# [END] symbol: WY / total_time: 38.63[sec]
# [START] symbol: WHR
# [END] symbol: WHR / total_time: 36.57[sec]
# [START] symbol: WMB
# [END] symbol: WMB / total_time: 38.17[sec]
# [START] symbol: WLTW
# [END] symbol: WLTW / total_time: 20.77[sec]
# [START] symbol: WYNN
# [END] symbol: WYNN / total_time: 36.48[sec]
# [START] symbol: XEL
# [END] symbol: XEL / total_time: 38.89[sec]
# [START] symbol: XRX
# [END] symbol: XRX / total_time: 37.56[sec]
# [START] symbol: XLNX
# [END] symbol: XLNX / total_time: 37.66[sec]
# [START] symbol: XYL
# [END] symbol: XYL / total_time: 29.38[sec]
# [START] symbol: YUM
# [END] symbol: YUM / total_time: 35.77[sec]
# [START] symbol: ZBRA
# [END] symbol: ZBRA / total_time: 37.37[sec]
# [START] symbol: ZBH
# [END] symbol: ZBH / total_time: 34.86[sec]
# [START] sym

In [28]:
index = 4
rules = [macrotrends_scrapying_rules[index]]
category = rules[0]["CATEGORY"]

## 実行
for i in range(443, len(s_and_p_stock_list)):
    df = df.append(addMacrotrendsInfoBySymbol(s_and_p_stock_list[i], driver, rules))
df.to_csv("./output/macrotrends_{0}.csv".format(category))

# [START] symbol: DIS
# [END] symbol: DIS / total_time: 64.20[sec]
# [START] symbol: TMO
# [END] symbol: TMO / total_time: 50.96[sec]
# [START] symbol: TIF
# [END] symbol: TIF / total_time: 50.55[sec]
# [START] symbol: TJX
# [END] symbol: TJX / total_time: 51.50[sec]
# [START] symbol: TSCO
# [END] symbol: TSCO / total_time: 52.62[sec]
# [START] symbol: TT
# [END] symbol: TT / total_time: 52.37[sec]
# [START] symbol: TDG
# [END] symbol: TDG / total_time: 52.61[sec]
# [START] symbol: TFC
# [END] symbol: TFC / total_time: 55.27[sec]
# [START] symbol: TWTR
# [END] symbol: TWTR / total_time: 34.79[sec]
# [START] symbol: TYL
# [END] symbol: TYL / total_time: 53.93[sec]
# [START] symbol: TSN
# [END] symbol: TSN / total_time: 53.83[sec]
# [START] symbol: USB
# [END] symbol: USB / total_time: 54.95[sec]
# [START] symbol: UDR
# [END] symbol: UDR / total_time: 55.43[sec]
# [START] symbol: ULTA
# [END] symbol: ULTA / total_time: 50.10[sec]
# [START] symbol: UAA
# [END] symbol: UAA / total_time: 53